# Unicode Visualization of the Bengali Alphabet

This notebook attempts to extend upon this to visualize the Bengali Alphabet, and builds on my previous [Bengali AI Dataset - EDA Grapheme Combinations](https://www.kaggle.com/jamesmcguigan/bengali-ai-dataset-eda-grapheme-combinations/)

Unicode itself is encoded as a multibyte string, using a lower level of base_graphemes than root/vowel/consonant diacritics. Some Benglai Graphemes have multiple renderings for the same root/vowel/consonant combination, which is implemented in unicode by allowing duplicate base_graphemes within the encoding. 

* This potentually opens up another datasource for investigation, which is to explore the full range of diacritic combinations within the unicode specification. The paper [Fonts-2-Handwriting: A Seed-Augment-Train framework for universal digit classification](https://arxiv.org/pdf/1905.08633.pdf) also makes the suggestion that it may be possible to generate synethetic data for handwriting recognition by rendering each of the unicode graphemes using various Bengali fonts

## Imports

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from IPython.display import Markdown, HTML
from collections import Counter
from itertools import chain
from functools import reduce
# from src.jupyter import grid_df_display, combination_matrix

pd.set_option('display.max_columns',   500)
pd.set_option('display.max_colwidth',   -1 if os.environ.get('KAGGLE_KERNEL_RUN_TYPE','') else None)

%load_ext autoreload
%autoreload 2

In [2]:
# Source: https://github.com/JamesMcGuigan/kaggle-digit-recognizer/blob/master/src/utils/confusion_matrix.py
from typing import Union

import pandas as pd
from pandas.io.formats.style import Styler


def combination_matrix(dataset: pd.DataFrame, x: str, y: str, z: str,
                       format=None, unique=True) -> Union[pd.DataFrame, Styler]:
    """
    Returns a combination matrix, showing all valid combinations between three DataFrame columns.
    Sort of like a heatmap, but returning lists of (optionally) unique values

    :param dataset: The dataframe to create a combination_matrx from
    :param x: column name to use for the X axis
    :param y: column name to use for the Y axis
    :param z: column name to use for the Z axis (values that appear in the cells)
    :param format: '', ', '-', ', '\n'    = format value lists as "".join() string
                    str, bool, int, float = cast value lists
    :param unique:  whether to return only unique values or not - eg: combination_matrix(unique=False).applymap(sum)
    :return: returns nothing
    """
    unique_y = sorted(dataset[y].unique())
    combinations = pd.DataFrame({
        n: dataset.where(lambda df: df[y] == n)
            .groupby(x)[z]
            .pipe(lambda df: df.unique() if unique else df )
            .apply(list)
            .apply(sorted)
        for n in unique_y
    }).T

    if isinstance(format, str):
        combinations = combinations.applymap(
            lambda cell: f"{format}".join([str(value) for value in list(cell) ])
            if isinstance(cell, list) else cell
        )
    if format == str:   combinations = combinations.applymap(lambda cell: str(cell)      if isinstance(cell, list) and len(cell) > 0 else ''     )
    if format == bool:  combinations = combinations.applymap(lambda cell: True           if isinstance(cell, list) and len(cell) > 0 else False  )
    if format == int:   combinations = combinations.applymap(lambda cell: int(cell[0])   if isinstance(cell, list) and len(cell)     else ''     )
    if format == float: combinations = combinations.applymap(lambda cell: float(cell[0]) if isinstance(cell, list) and len(cell)     else ''     )

    combinations.index.rename(y, inplace=True)
    combinations.fillna('', inplace=True)
    if format == '\n':
        return combinations.style.set_properties(**{'white-space': 'pre-wrap'})  # needed for display
    else:
        return combinations  # Allows for subsequent .applymap()

# Statistics

We can decode the graphemes into their constituant base_graphemes using `list()` 

In [3]:
dataset = pd.read_csv('../input/bengaliai-cv19/train.csv'); 
dataset['base_graphemes'] = dataset['grapheme'].apply(list)
dataset.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,base_graphemes
0,Train_0,15,9,5,ক্ট্রো,"[ক, ্, ট, ্, র, ো]"
1,Train_1,159,0,0,হ,[হ]
2,Train_2,22,3,5,খ্রী,"[খ, ্, র, ী]"
3,Train_3,53,2,2,র্টি,"[র, ্, ট, ি]"
4,Train_4,71,9,5,থ্রো,"[থ, ্, র, ো]"


There are 62 base_graphemes in the unicode encoding, with a median of 4 and maximum of 8 symbols required to encode each grapheme. 

We can also see the percentage frequency for each root diacritic.

In [4]:
base_diacritics_unique = sorted(set(chain(*dataset['base_graphemes'].values)))
base_diacritics_stats  = {
    "mean":   round( dataset['base_graphemes'].apply(len).mean(), 2),
    "median": np.median( dataset['base_graphemes'].apply(len) ),
    "min":    dataset['base_graphemes'].apply(len).min(),
    "max":    dataset['base_graphemes'].apply(len).max(),
    "std":    dataset['base_graphemes'].apply(len).std(),    
    "unique": len( set(chain(*dataset['base_graphemes'].values))),
    "count":  len(list(chain(*dataset['base_graphemes'].values))),
    "mean_duplicated_bases":  dataset['base_graphemes'].apply(lambda value: (len(value) - len(set(value)))).mean(),
    "max_duplicated_bases":   dataset['base_graphemes'].apply(lambda value: (len(value) - len(set(value)))).max(),    
    "count_duplicated_bases": dataset['base_graphemes'].apply(lambda value: (len(value) - len(set(value))) != 0).sum(),        
}
base_diacritics_counter = dict( 
    sum(dataset['base_graphemes'].apply(Counter), Counter()).most_common()
)

display( pd.DataFrame([base_diacritics_counter]) / base_diacritics_stats['count'] )
display( " ".join(base_diacritics_unique) )
display( base_diacritics_stats )

,্,র,া,ে,ি,য,ন,ব,স,ত,ু,ক,দ,ম,ী,ো,ল,ট,প,ষ,গ,জ,শ,ঁ,ধ,ড,ভ,চ,ফ,থ,ণ,ূ,হ,ৃ,ঙ,ঠ,ঘ,ৌ,ৈ,খ,ছ,ঞ,য়,ঝ,ঢ,ড়,এ,অ,ই,আ,উ,ঢ়,ঈ,‍,ৎ,ঋ,ঊ,ঐ,ও,ং,ঔ,ঃ
0,0.236251,0.070838,0.052907,0.041198,0.037245,0.036703,0.034193,0.027519,0.027404,0.027244,0.027034,0.025736,0.024474,0.024326,0.023167,0.022995,0.021453,0.019388,0.019057,0.014875,0.012952,0.012119,0.011734,0.010649,0.01041,0.010237,0.010096,0.010056,0.008209,0.007773,0.007735,0.007598,0.006744,0.006219,0.005869,0.005425,0.00521,0.005111,0.00506,0.00493,0.004798,0.00416,0.002273,0.001974,0.001843,0.001638,0.000637,0.000483,0.000475,0.000456,0.000442,0.00044,0.000251,0.000242,0.000235,0.000225,0.000219,0.000218,0.000217,0.000211,0.000209,0.000208


'ঁ ং ঃ অ আ ই ঈ উ ঊ ঋ এ ঐ ও ঔ ক খ গ ঘ ঙ চ ছ জ ঝ ঞ ট ঠ ড ঢ ণ ত থ দ ধ ন প ফ ব ভ ম য র ল শ ষ স হ া ি ী ু ূ ৃ ে ৈ ো ৌ ্ ৎ ড় ঢ় য় \u200d'

{'mean': 3.47,
 'median': 4.0,
 'min': 1,
 'max': 8,
 'std': 1.1731124865863312,
 'unique': 62,
 'count': 697187,
 'mean_duplicated_bases': 0.1691794463254332,
 'max_duplicated_bases': 2,
 'count_duplicated_bases': 31276}

# Base Graphemes

As there are fewer base_graphemes than root_graphemes in the Bengali alphabet. Thus we can perform a set analyis to determine how the grapheme_roots are themselves decomposed into a lower level of root diacritic combinations.

In [5]:
base_diacritic_sets = {
    key: dataset.groupby(key)['base_graphemes']
                .apply(lambda group: reduce(lambda a,b: set(a) & set(b), group)) 
                .apply(sorted)     
    for key in [ 'vowel_diacritic', 'consonant_diacritic', 'grapheme_root' ]
}
display(
    pd.DataFrame(base_diacritic_sets)
        .applymap(lambda x: x if x is not np.nan else set())        
        .applymap(lambda group: "\n".join(group))
        .T
        .style.set_properties(**{'white-space': 'pre-wrap'})
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167
vowel_diacritic,,া,ি,ী,ু,ূ,ৃ,ে,ৈ,ো,ৌ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
consonant_diacritic,,ঁ,র ্,য র ্,য ্,র ্,য র ্,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
grapheme_root,ং,ঃ,অ,আ,ই,ঈ,উ,ঊ,ঋ,এ,ঐ,ও,ঔ,ক,ক ্,ক ট ্,ক ত ্,ক ল ্,ক ষ ্,ক ণ ষ ্,ক ম ষ ্,ক স ্,খ,গ,গ ধ ্,গ ন ্,গ ব ে ্,গ ম ্,গ ল ্,ঘ,ঘ ন ্,ঙ,ক ঙ ্,ক ঙ ত ি ্,ক ঙ ষ ্,খ ঙ ্,গ ঙ ্,ঘ ঙ ্,চ,চ ্,চ ছ ্,চ ছ ব ্,ছ,জ,জ ্,জ ব ্,জ ঞ ্,জ ব ্,ঝ,ঞ,চ ঞ ্,ছ ঞ ্,জ ঞ ্,ট,ট ্,ঠ,ড,ড ্,ঢ,ণ,ট ণ ্,ঠ ণ ্,ড ণ ্,ণ ্,ত,ত ্,ত ব ্,ত থ ্,ত ন ্,ত ব ্,ত ম ্,থ,দ,ঘ দ া ্,দ ্,দ ধ ্,দ ব ্,দ ভ ্,দ ম ্,ধ,ধ ব ্,ন,জ ন ্,ট ন ্,ঠ ন ্,ড ন ্,ত ন ্,ত ন ব ্,থ ন ্,দ ন ্,দ ন ব ্,ধ ন ্,ন ্,ন ব ্,ন ম ্,ন স ্,প,ট প ্,ত প ্,ন প ্,প ্,প ল ্,প স ্,ফ,ট ফ ্,ফ ্,ফ ল ্,ব,জ ব ি ্,দ ব ্,ধ ব ্,ব ্,ব ল ্,ভ,ভ ল া ্,ম,ন ম ্,প ম ্,ব ম ্,ভ ম ্,ম ্,ম ল ্,য,র,ল,ক ল ্,গ ল ু ্,ট ল ্,ড ল ্,প ল ্,ব ল ্,ম ল ্,ল ্,শ,চ শ ্,ন শ ্,ব শ ্,ম শ ্,ল শ ্,ষ,ক ষ ্,ট ষ ্,ঠ ষ ্,ণ ষ ্,প ষ ্,ফ ষ ্,ম ষ ্,স,ক স ্,ট স ্,ত স ্,থ স ্,ন স ্,প স ্,ফ স ্,ব স ্,ম স ্,ল স ্,স ু ্,হ,ন হ ্,ব হ ্,ম হ ্,ল হ া ্,ৎ,ড়,ঢ়,য়


If we extend the dataset with the information, we can display a combination matrix in pure Bengali

In [6]:
for key in [ 'vowel_diacritic', 'consonant_diacritic', 'grapheme_root' ]:
    base_key = key.split('_')[0] + '_base'
    zfill = 3 if key == 'grapheme_root' else 2
    dataset[base_key] = (
        dataset[key]
            .apply(lambda value: [ str(value).zfill(zfill)] + sorted(base_diacritic_sets[key][value]))
            .apply(lambda value: " ".join(value))            
            .fillna('')
    )
# Make numeric strings sortable
dataset.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,base_graphemes,vowel_base,consonant_base,grapheme_base
0,Train_0,15,9,5,ক্ট্রো,"[ক, ্, ট, ্, র, ো]",09 ো,05 র ্,015 ক ট ্
1,Train_1,159,0,0,হ,[হ],00,00,159 হ
2,Train_2,22,3,5,খ্রী,"[খ, ্, র, ী]",03 ী,05 র ্,022 খ
3,Train_3,53,2,2,র্টি,"[র, ্, ট, ি]",02 ি,02 র ্,053 ট
4,Train_4,71,9,5,থ্রো,"[থ, ্, র, ো]",09 ো,05 র ্,071 থ


# Visualization of Grapheme Combinations in Bengali Alphabet

# Vowel/Consonant Combinations Table

In [7]:
combination_matrix(dataset, x='consonant_base', y='vowel_base', z='grapheme', format=' ')

,00,01 ঁ,02 র ্,03 য র ্,04 য ্,05 র ্,06 য র ্
vowel_base,,,,,,,
00,ং ঃ অ আ ই ঈ উ ঊ ঋ এ ঐ ও ঔ ক ক্ক ক্ট ক্ত ক্ল ক্ষ ক্ষ্ণ ক্ষ্ম ক্স খ গ গ্ধ গ্ন গ্ম ঘ ঘ্ন ঙ ঙ্ক ঙ্খ ঙ্গ ঙ্ঘ চ চ্চ চ্ছ চ্ছ্ব ছ জ জ্জ জ্জ্ব জ্ঞ জ্ব ঝ ঞ ঞ্চ ঞ্ছ ঞ্জ ট ট্ট ঠ ড ড্ড ঢ ণ ণ্ট ণ্ঠ ণ্ড ণ্ণ ত ত্ত ত্ত্ব ত্থ ত্ন ত্ব ত্ম থ দ দ্দ দ্ধ দ্ব দ্ভ দ্ম ধ ধ্ব ন ন্ট ন্ঠ ন্ড ন্ত ন্ত্ব ন্থ ন্দ ন্দ্ব ন্ধ ন্ন ন্ব ন্ম ন্স প প্ট প্ত প্ন প্প প্ল প্স ফ ফ্ট ফ্ফ ফ্ল ব ব্দ ব্ধ ব্ব ব্ল ভ ম ম্ন ম্প ম্ব ম্ভ ম্ম ম্ল য র ল ল্ক ল্ট ল্ড ল্প ল্ব ল্ম ল্ল শ শ্চ শ্ন শ্ব শ্ম শ্ল ষ ষ্ক ষ্ট ষ্ঠ ষ্ণ ষ্প ষ্ফ ষ্ম স স্ক স্ট স্ত স্থ স্প স্ফ স্ব স্ম হ হ্ন হ্ব হ্ম ৎ ড় ঢ় য়,আঁ ইঁ উঁ এঁ পঁ,র্ক র্ক্স র্খ র্গ র্ঘ র্চ র্ছ র্জ র্ঝ র্ট র্ড র্ণ র্ত র্ত্ত র্থ র্দ র্দ্দ র্দ্ধ র্দ্ব র্দ্র র্ধ র্ধ্ব র্ন র্ন্ত র্প র্ফ র্ব র্ব্ব র্ভ র্ম র্ম্ম র্য র্ল র্ল্ড র্শ র্শ্ব র্ষ র্স র্স্ট র্হ,র্ঘ্য র্জ্য র্থ্য র্ব্য,ক্য ক্ষ্য খ্য গ্য চ্য জ্য ট্য ঠ্য ঢ্য ণ্য ত্ত্য ত্ম্য ত্য থ্য দ্ব্য দ্য ধ্য ন্ত্য ন্দ্য ন্ধ্য ন্য প্য ব্য ভ্য ম্য য্য ল্য শ্য ষ্ট্য ষ্ঠ্য ষ্য স্থ্য স্য হ্য,ক্ট্র ক্র গ্র ঘ্র জ্র ট্র ড্র ত্র দ্র ন্ড্র ন্ত্র ন্দ্র ন্ধ্র প্র ফ্র ব্র ভ্র ম্প্র ম্ভ্র ম্র শ্র ষ্ট্র স্ক্র স্ট্র স্ত্র স্র হ্র,ত্র্য দ্র্য
01 া,কা ক্কা ক্টা ক্তা ক্লা ক্ষা ক্সা খা গা গ্ধা গ্না গ্লা ঘা ঙা ঙ্কা ঙ্ক্ষা ঙ্খা ঙ্গা চা চ্চা চ্ছা চ্ছ্বা ছা জা জ্জা জ্ঞা জ্বা ঝা ঞা ঞ্চা ঞ্জা টা ট্টা ঠা ডা ড্ডা ঢা ণা ণ্টা ণ্ঠা ণ্ডা তা ত্তা ত্ত্বা ত্থা ত্বা ত্মা থা দা দ্ঘা দ্দা দ্ধা দ্বা দ্ভা দ্মা ধা না ন্টা ন্ডা ন্তা ন্থা ন্দা ন্ধা ন্না ন্মা ন্সা পা প্টা প্তা প্না প্পা প্লা ফা ফ্লা বা ব্দা ব্বা ব্লা ভা ভ্লা মা ম্না ম্পা ম্বা ম্ভা ম্মা ম্লা যা রা লা ল্কা ল্টা ল্ডা ল্পা ল্লা শা শ্চা শ্না শ্বা ষা ষ্কা ষ্টা ষ্ঠা ষ্ণা ষ্পা সা স্কা স্টা স্তা স্থা স্না স্পা স্বা স্মা স্লা হা হ্বা হ্মা হ্লা ড়া য়া,কাঁ খাঁ গাঁ ঘাঁ চাঁ ছাঁ ঝাঁ ঠাঁ তাঁ দাঁ ধাঁ পাঁ ফাঁ বাঁ ভাঁ রাঁ সাঁ হাঁ,র্কা র্খা র্গা র্ঘা র্চা র্ছা র্জা র্টা র্ডা র্ণা র্তা র্থা র্দা র্দ্দা র্ধা র্না র্বা র্ভা র্মা র্যা র্লা র্শা র্ষা র্সা র্স্থা,,অ্যা এ্যা ক্যা ক্ষ্যা খ্যা গ্ন্যা গ্যা গ্ল্যা ঘ্যা চ্যা ছ্যা জ্যা ট্যা ঠ্যা ড্যা ণ্যা ত্যা থ্যা দ্যা ধ্যা ন্ধ্যা ন্ন্যা ন্যা প্যা প্ল্যা ফ্যা ফ্ল্যা ব্যা ব্ল্যা ভ্যা ম্যা য্যা র‍্যা ল্যা শ্যা ষ্ট্যা ষ্যা স্ক্যা স্ট্যা স্প্যা স্যা হ্যা য়্যা,ক্রা গ্রা ঘ্রা ট্রা ড্রা ত্রা দ্ভ্রা দ্রা ন্ট্রা ন্ত্রা ন্দ্রা প্রা ফ্রা ব্রা ভ্রা ম্ভ্রা ম্রা শ্রা ষ্প্রা স্ট্রা স্ত্রা স্প্রা স্রা হ্রা,ক্র্যা গ্র্যা ট্র্যা প্র্যা ফ্র্যা ব্র্যা স্ট্র্যা
02 ি,কি ক্কি ক্টি ক্তি ক্লি ক্ষি ক্সি খি গি গ্নি গ্মি গ্লি ঘি ঘ্নি ঙি ঙ্কি ঙ্ক্তি ঙ্ক্ষি ঙ্খি ঙ্গি ঙ্ঘি চি চ্চি চ্ছি ছি জি জ্জি জ্বি ঝি ঞ্চি ঞ্ছি ঞ্জি টি ট্টি ঠি ডি ড্ডি ঢি ণি ণ্ঠি ণ্ডি তি ত্তি ত্ত্বি ত্নি ত্বি ত্মি থি দি দ্দি দ্ধি দ্বি দ্ভি ধি নি ন্টি ন্ঠি ন্ডি ন্তি ন্থি ন্দি ন্দ্বি ন্ধি ন্নি ন্বি ন্মি ন্সি পি প্তি প্লি ফি বি ব্জি ব্দি ব্ধি ব্বি ব্লি ভি মি ম্পি ম্বি ম্ভি ম্মি যি রি লি ল্কি ল্টি ল্ডি ল্পি ল্মি ল্লি শি শ্চি শ্বি শ্মি শ্লি ষি ষ্টি ষ্ঠি সি স্কি স্টি স্তি স্থি স্নি স্পি স্বি স্মি হি হ্নি ড়ি য়ি,ছিঁ পিঁ সিঁ,র্কি র্গি র্চি র্জি র্টি র্ডি র্ণি র্তি র্থি র্দি র্দ্বি র্ধি র্নি র্পি র্ফি র্বি র্ভি র্মি র্লি র্শি র্শ্বি র্ষি র্সি র্হি,,গ্যি জ্যি ত্যি দ্যি ধ্যি ব্যি শ্যি হ্যি,ক্রি খ্রি গ্রি ট্রি ড্রি ত্রি থ্রি দ্রি ন্ট্রি ন্ত্রি ন্দ্রি প্রি ফ্রি ব্রি শ্রি ষ্ক্রি স্ক্রি স্ট্রি স্ত্রি স্প্রি,
03 ী,কী ক্ষী ক্ষ্মী খী গী গ্নী ঙ্কী ঙ্গী চী জী জ্জী জ্ঞী ঞ্জী টী ঠী ডী ড্ডী ণী ণ্ডী তী ত্তী ত্ত্বী ত্নী ত্বী ত্মী থী দী দ্দী দ্বী ধী নী ন্জী ন্ডী ন্তী ন্থী ন্দী ন্দ্বী ন্ধী ন্নী ন্বী ন্সী পী প্পী ফী বী ব্দী ভী মী ম্নী ম্বী ম্ভী ম্মী যী রী লী ল্কী ল্পী ল্লী শী শ্মী শ্লী ষী ষ্টী ষ্ঠী সী স্টী স্তী স্পী স্ফী স্বী স্মী হী হ্মী ড়ী য়ী,,র্কী র্ক্সী র্গী র্জী র্ণী র্তী র্ত্তী র্ত্রী র্থী র্দী র্নী র্মী র্শী র্ষী র্সী,,,ক্রী খ্রী গ্রী ত্রী দ্রী ন্ত্রী ন্দ্রী প্রী ফ্রী ব্রী ম্প্রী শ্রী ষ্ক্রী ষ্ট্রী স্ট্রী স্ত্রী,
04 ু,কু ক্ষু খু গু গ্লু ঘু ঙু ঙ্কু ঙ্গু চু চ্চু চ্ছু ছু জু জ্জু ঝু ঞ্চু ঞ্জু টু ঠু ডু ঢু ণু ণ্টু ণ্ডু তু থু দু দ্দু দ্বু দ্ভু ধু নু ন্জু ন্টু ন্ডু ন্তু ন্দু ন্ধু ন্নু ন্মু পু প্লু ফু বু ব্দু ব্বু ব্লু ভু মু ম্পু ম্বু ম্ভু ম্মু যু রু লু ল্গু ল্টু ল্লু শু শ্চু শ্লু ষু ষ্টু ষ্ঠু ষ্ণু সু স্কু স্টু স্তু স্ফু স্সু হু ড়ু য়ু,কুঁ খুঁ গুঁ ছুঁ ঝুঁ পুঁ ফুঁ হুঁ,র্কু র্গু র্ঘু র্জু র্টু র্

# Base/Vowel Combinations Table

In [8]:
combination_matrix(dataset, x='grapheme_base', y='vowel_base', z='grapheme', format='\n')

# Base/Consonant Combinations Table

In [9]:
combination_matrix(dataset, x='grapheme_base', y='consonant_base', z='grapheme', format='\n')

# Full Combination Table

In [10]:
combination_matrix(dataset, x=['vowel_base','consonant_base'], y='grapheme_base', z='grapheme', format=' ').T

grapheme_base             000 ং 001 ঃ 002 অ 003 আ 004 ই 005 ঈ 006 উ 007 ঊ  \
vowel_base consonant_base                                                   
00         00              ং     ঃ     অ     আ     ই     ঈ     উ     ঊ      
           01 ঁ                              আঁ    ইঁ          উঁ           
           02 র ্                                                           
           03 য র ্                                                         
           04 য ্                                                           
           05 র ্                                                           
           06 য র ্                                                         
01 া       00                                                               
           01 ঁ                                                             
           02 র ্                                                           
           04 য ্                      অ্যা                                 
           05 র ্                                                           
           06 য র ্                                                         
02 ি       00                                                               
           01 ঁ                                                             
           02 র ্                                                           
           04 য ্                                                           
           05 র ্                                                           
03 ী       00                                                               
           02 র ্                                                           
           05 র ্                                                           
04 ু       00                                                               
           01 ঁ                                                             
           02 র ্                                                           
           04 য ্                                                           
           05 র ্                                                           
05 ূ       00                                                               
           02 র ্                                                           
           04 য ্                                                           
           05 র ্                                                           
06 ৃ       00                                                               
           02 র ্                                                           
07 ে       00                                                               
           01 ঁ                                                             
           02 র ্                                                           
           04 য ্                                                           
           05 র ্                                                           
08 ৈ       00                                                               
           04 য ্                                                           
           05 র ্                                                           
09 ো       00                                                               
           01 ঁ                                                             
           02 র ্                                                           
           04 য ্                                                           
           05 র ্                                                           
10 ৌ       00                                                               
           01 ঁ                                                             
           05 র ্                                                           

grapheme_base             008 ঋ 009 এ 010 ঐ 011 ও 012 ঔ   013 ক 014 ক ্  \
vowel_base consonant_base                                                 

In [11]:
combination_matrix(dataset, x=['vowel_base','consonant_base'], y='grapheme_base', z='grapheme', format=' ')

vowel_base     00                                              01 া       \
consonant_base 00 01 ঁ 02 র ্ 03 য র ্ 04 য ্ 05 র ্ 06 য র ্    00 01 ঁ   
grapheme_base                                                              
000 ং           ং                                                          
001 ঃ           ঃ                                                          
002 অ           অ                                                          
003 আ           আ  আঁ                                                      
004 ই           ই  ইঁ                                                      
...            .. ..   ..     ..       ..     ..     ..       ..    ..     
163 ল হ া ্                                                    হ্লা        
164 ৎ           ৎ                                                          
165 ড়           ড়                                              ড়া          
166 ঢ়           ঢ়                                                          
167 য়           য়                                              য়া          

vowel_base                                   02 ি                            \
consonant_base 02 র ্ 04 য ্ 05 র ্ 06 য র ্   00 01 ঁ 02 র ্ 04 য ্ 05 র ্   
grapheme_base                                                                 
000 ং                                                                         
001 ঃ                                                                         
002 অ                  অ্যা                                                   
003 আ                                                                         
004 ই                                                                         
...            ..       ...  ..     ..       ..   ..   ..     ..     ..       
163 ল হ া ্                                                                   
164 ৎ                                                                         
165 ড়                                         ড়ি                              
166 ঢ়                                                                         
167 য়                  য়্যা                   য়ি                              

vowel_base     03 ী               04 ু                           05 ূ         \
consonant_base   00 02 র ্ 05 র ্   00 01 ঁ 02 র ্ 04 য ্ 05 র ্   00 02 র ্   
grapheme_base                                                                  
000 ং                                                                          
001 ঃ                                                                          
002 অ                                                                          
003 আ                                                                          
004 ই                                                                          
...            ..   ..     ..     ..   ..   ..     ..     ..     ..   ..       
163 ল হ া ্                                                                    
164 ৎ                                                                          
165 ড়           ড়ী                 ড়ু                                          
166 ঢ়                                                                          
167 য়           য়ী                 য়ু                             য়ূ           

vowel_base                   06 ৃ        07 ে                           08 ৈ  \
consonant_base 04 য ্ 05 র ্   00 02 র ্   00 01 ঁ 02 র ্ 04 য ্ 05 র ্   00   
grapheme_base                                                                  
000 ং                                                                          
001 ঃ                                                                          
002 অ                                                                          
003 আ                                                                          
004 ই                                                                          
...            ..     ..     ..   ..     ..   ..   ..     ..     ..